In [51]:
from datasets import load_dataset, load_from_disk, Audio
import os

In [52]:
dataset = load_from_disk("data-chatbot-arena-spoken-style-11labs")
print("len(dataset):", len(dataset))

len(dataset): 654


In [53]:
dataset[0]

{'id': 5434,
 'original_id': '15793788be324bd588e7b9707615c279',
 'winner_content': 'model_b',
 'winner_style': 'model_b',
 'question_original': 'write a blog about eLearning',
 'assistant_a': 'The world of eLearning is rapidly growing, and it is becoming increasingly important for businesses to stay competitive. As more people turn to eLearning as a way to learn new skills, companies must find ways to make their eLearning programs engaging and effective. To do this, companies must focus on creating courses that are visually appealing and easy to navigate, while also providing engaging content that keeps learners engaged and motivated. Additionally, companies must ensure that their eLearning platform is secure, allowing learners to access their courses from any device with an internet connection. By investing in these strategies, companies can create eLearning programs that are both effective and engaging, ensuring that their employees are equipped with the skills they need to succeed.

In [54]:
def add_questions_refined(ex):
    id = ex['id']
    # refined_question_path = f"./refined_questions_v1.5/{id}.refined_question.txt"
    # refined_question_path = f"./refined_questions_v1.6/{id}.refined_question.txt"
    refined_question_path = f"./refined_questions_v1.7/{id}.refined_question.txt"
    with open(refined_question_path) as f:
        refined_question = f.read().strip('"').strip()

    # None if rejected
    # path = f"./refined_questions_v1.5/verdict/{id}.verdict.txt"
    # path = f"./refined_questions_v1.6/verdict/{id}.verdict.txt"
    path = f"./refined_questions_v1.7/verdict/{id}.verdict.txt"
    # check if file exists
    if not os.path.exists(path):
        ex['question_refined'] = None
        # print(path, "not found")
        return ex
    
    with open(path) as f:
        x = f.read().strip()
    if x == "VALID":
        ex['question_refined'] = refined_question
    elif x == "INVALID":
        ex['question_refined'] = None
    else:
        raise Exception()
    return ex
ds = ds.map(add_questions_refined)
ds = ds.filter(lambda x: x['question_refined'] is not None)
print("len(ds):", len(ds))

Map:   0%|          | 0/570 [00:00<?, ? examples/s]

Filter:   0%|          | 0/570 [00:00<?, ? examples/s]

len(ds): 570


In [55]:
def add_question_refined_wav(ex):
    id = ex['id']
    # path = f"./refined_questions_kokoro_wav_v1.5/{id}.wav"
    path = f"./refined_questions_kokoro_wav_v1.7/{id}.wav"
    ex['question_refined_wav'] = path
    return ex
ds = ds.map(add_question_refined_wav)


Map:   0%|          | 0/570 [00:00<?, ? examples/s]

In [56]:
def add_model_type(ex):
    ex['model_type'] = ex['winner_style']
    return ex
ds = ds.map(add_model_type)

Map:   0%|          | 0/570 [00:00<?, ? examples/s]

In [57]:
def add_assistant_wav(ex):
    id = ex['id']
    winner = ex['winner_style']
    if winner == 'model_a':
        path = f"./generated_11labs_audio/{id}_model_a.11labs.wav"
    elif winner == 'model_b':    
        path = f"./generated_11labs_audio/{id}_model_b.11labs.wav"
    else:
        raise Exception()
    if os.path.exists(path):
        ex['assistant_wav'] = path
    else:
        ex['assistant_wav'] = None
    return ex
ds = ds.map(add_assistant_wav)
ds = ds.filter(lambda x: x['assistant_wav'] is not None)

Map:   0%|          | 0/570 [00:00<?, ? examples/s]

Filter:   0%|          | 0/570 [00:00<?, ? examples/s]

In [58]:
print("len(ds):", len(ds))

len(ds): 570


In [59]:
ds_filtered = ds.select_columns(
    ["id", "original_id", "model_type", "question_refined", "question_refined_wav", "assistant_wav"]
).cast_column("question_refined_wav", Audio()).cast_column("assistant_wav", Audio())

In [60]:
ds_filtered

Dataset({
    features: ['id', 'original_id', 'model_type', 'question_refined', 'question_refined_wav', 'assistant_wav'],
    num_rows: 570
})

In [23]:
# from huggingface_hub import login
# login(token="")

In [61]:
ds_filtered.push_to_hub("potsawee/chatbot-arena-spoken-style-eval-570", max_shard_size="100MB")

Uploading the dataset shards:   0%|          | 0/10 [00:00<?, ?it/s]

Map:   0%|          | 0/57 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Map:   0%|          | 0/57 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Map:   0%|          | 0/57 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Map:   0%|          | 0/57 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/potsawee/chatbot-arena-spoken-style-eval-570/commit/443c9d1af64bca05234923eb8d9774251d8b59d1', commit_message='Upload dataset', commit_description='', oid='443c9d1af64bca05234923eb8d9774251d8b59d1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/potsawee/chatbot-arena-spoken-style-eval-570', endpoint='https://huggingface.co', repo_type='dataset', repo_id='potsawee/chatbot-arena-spoken-style-eval-570'), pr_revision=None, pr_num=None)